In [0]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [0]:
revs= pd.read_csv('toronto_reviews.csv')

In [0]:
del revs['Unnamed: 0']

In [4]:
revs.head()

,business_id,date,elem,review_id,stars_revs,user_id,biz_num,address,categories,hours,latitude,longitude,name,postal_code,review_count,stars_bus,user_stars_mean,user_review_num
0,AakkkTuGZA2KBodKi2_u8A,2012-07-16 00:37:14,10766,JVcjMhlavKKn3UIt9p9OXA,1.0,TpyOT5E16YASd7EWjLQlrw,943,1603 Dundas Street W,"Asian Fusion, Restaurants, Vietnamese","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",43.649674,-79.435116,Pho Phuong,M6K 1T9,54,3.346230,3.674419,44
1,YvrylyuWgbP90RgMqZQVnQ,2017-04-07 21:27:49,33371,svK3nBU7Rk8VfGorlrN52A,5.0,NJlxGtouq06hhC7sS2ECYw,2934,232 Wellington Street W,"Sandwiches, Restaurants, Italian, Pizza","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",43.645277,-79.389704,Jz's Pizza,M5V 3W1,70,3.942857,1.000000,2
2,y-Iw6dZflNix4BdwIyTNGA,2014-06-27 21:19:23,57511,4bUyL7lzoWzDZaJETAKREg,3.0,_N7Ndn29bpll_961oPeEfw,5099,2876 Dundas Street W,"Restaurants, Food, Breweries, Gastropubs, Pizza","{'Monday': '17:0-1:0', 'Tuesday': '12:0-1:0', ...",43.665441,-79.465238,Indie Ale House,M6P 1Y9,234,4.087616,3.625000,9
3,YSUcHqlKMPHHJ_cTrqtNrA,2017-12-29 13:55:19,32926,E6B-2U2sGG3xgmnNWZAEew,1.0,DbccYu3OppWKl21OanZnTg,2894,32 Wellington Street E,"Italian, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",43.648404,-79.375469,Vagabondo Italian Ristorante+Lounge,M5E 1C6,76,3.117240,3.250000,13
4,TVLAvAFx0FqAkKGKHVjHpQ,2017-12-15 22:49:01,28247,6JgzKIDvophsU1yfK0DdyA,4.0,mZ1gXzL6Tn5Oky8_j0Kp7g,2479,140 Spadina Avenue,"Soup, Juice Bars & Smoothies, Restaurants, Foo...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",43.647726,-79.396247,Freshii,M5V 2L4,39,2.949343,3.625000,57


In [30]:
#Sumamry statistics about reviews, 305000 in total
revs.describe()

,elem,stars_revs,biz_num,latitude,longitude,review_count,stars_bus,user_stars_mean,user_review_num
count,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000
mean,30245.505143,3.649632,2670.604090,43.673505,-79.390846,220.967284,3.649632,3.649632,32.456466
std,16963.219643,1.263013,1501.965162,0.042646,0.046761,279.705114,0.527701,0.674939,61.321050
min,37.000000,1.000000,1.000000,43.592327,-79.703390,2.000000,0.500000,1.000000,1.000000
25%,15489.000000,3.000000,1382.000000,43.648752,-79.410044,60.000000,3.340359,3.419355,3.000000
50%,31032.000000,4.000000,2719.000000,43.656345,-79.391912,133.000000,3.701574,3.585366,10.000000
75%,44831.000000,5.000000,3961.000000,43.673749,-79.378802,272.000000,4.019922,4.000000,36.000000
max,59372.000000,5.000000,5253.000000,43.876501,-79.139353,2120.000000,5.500000,5.000000,717.000000


In [22]:
#Distribution of star ratings, 4 and 5 most common, 1 and 2 least common
revs.stars_revs.value_counts()/len(revs)

4.0    0.326349
5.0    0.304542
3.0    0.176755
2.0    0.098905
1.0    0.093449
Name: stars_revs, dtype: float64

In [25]:
#5000 businesses
revs.business_id.nunique()

5253

In [24]:
#Top business has 2000 reviews
revs.business_id.value_counts()[:10]

r_BrIgzYcwo1NAuG9dLbpg    2177
aLcFhMe6DDJ430zelCpd2A    1467
RtUvSWO_UZ8V3Wpj0n077w    1425
iGEvDk6hsizigmXhDKs2Vg    1183
N93EYZy9R0sdlEvubu94ig    1078
RwRNR4z3kY-4OsFqigY5sw     973
zgQHtqX0gqMw1nlBZl2VnQ     930
-av1lZI1JDY_RZN2eTMnWg     903
B70iTJjcPkuYn8ouUewWgw     864
e41TP5cXZqSrz50xCBJqZw     857
Name: business_id, dtype: int64

In [27]:
#80,000 unique reviewers
revs.user_id.nunique()

79775

In [29]:
#Most prolific user has given 700 reviews
revs.user_id.value_counts()[:10]

CxDOIDnH8gp9KXzpBHJYXw    717
Q9mA60HnY87C1TW5kjAZ6Q    424
O3pSxv1SyHpY4qi4Q16KzA    404
ic-tyi1jElL_umxZVh8KNA    323
1fNQRju9gmoCEvbPQBSo7w    317
iRQ_YKpCBdaCwvc2X8_3NQ    304
FREeRQtjdJU83AFtdETBBw    296
gwIqbXEXijQNgdESVc07hg    287
0BBUmH7Krcax1RZgbH4fSA    283
yT_QCcnq-QGipWWuzIpvtw    270
Name: user_id, dtype: int64

In [0]:
#In the next few cells I run a simple linear regression model, using mainly average stars of business and reviewer to try and guess rating

revs_num=revs[revs.describe().columns]

In [0]:
X= revs_num.drop(columns='stars_revs')

In [7]:
X.head()

,elem,biz_num,latitude,longitude,review_count,stars_bus,user_stars_mean,user_review_num
0,10766,943,43.649674,-79.435116,54,3.346230,3.674419,44
1,33371,2934,43.645277,-79.389704,70,3.942857,1.000000,2
2,57511,5099,43.665441,-79.465238,234,4.087616,3.625000,9
3,32926,2894,43.648404,-79.375469,76,3.117240,3.250000,13
4,28247,2479,43.647726,-79.396247,39,2.949343,3.625000,57


In [0]:
#Created a feature called 'star_est', which gives a weighted average of business and user average rating
#Applied smoothing on small samples by adding in 2 ratings of the dataset average, 3.65 stars (adding to 7.3) to each 
X['star_est']= (X.stars_bus * X.review_count + 7.3) / (X.review_count+2)**.5
X['star_est']+=(X.user_stars_mean*X.user_review_num + 7.3)/ (X.user_review_num+2)**.5
X['star_est']/= (X.user_review_num+2)**.5 + (X.review_count + 2)**.5
   

In [0]:
y= pd.DataFrame(revs.stars_revs)

In [0]:
lr= LinearRegression()

In [77]:
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
v= lr.predict(X)

In [0]:
y['predi']= v

In [80]:
y.head()

,stars_revs,predi
0,1.0,3.435789
1,5.0,3.194611
2,3.0,4.083431
3,1.0,2.981112
4,4.0,3.160578


In [0]:
y['predi_buck']= pd.cut(y['predi'], [0, 1,1.5, 2, 2.5, 3, 3.5, 4, 4.25, 4.5, 4.75, 5, 6])

In [82]:
#Seeing how many predictions are in each bucket, below 1 and above 5 is clearly absurd as scale is 1-5
y.predi_buck.value_counts()

(3.5, 4.0]     110443
(3.0, 3.5]      77984
(4.0, 4.25]     43881
(2.5, 3.0]      26642
(4.25, 4.5]     25568
(4.5, 4.75]      9896
(2.0, 2.5]       6209
(4.75, 5.0]      2467
(1.5, 2.0]       1440
(5.0, 6.0]        339
(1.0, 1.5]        275
(0.0, 1.0]         29
Name: predi_buck, dtype: int64

In [83]:
#Seeing actual average by prediction bucket

y.groupby('predi_buck').mean()

,stars_revs,predi
predi_buck,,
"(0.0, 1.0]",1.310345,0.848657
"(1.0, 1.5]",1.585455,1.330329
"(1.5, 2.0]",1.794444,1.804154
"(2.0, 2.5]",2.237558,2.311002
"(2.5, 3.0]",2.790781,2.803332
"(3.0, 3.5]",3.283058,3.283009
"(3.5, 4.0]",3.761298,3.751940
"(4.0, 4.25]",4.140995,4.116553
"(4.25, 4.5]",4.345745,4.361981


In [0]:
#Adjusting predictions to be between 1.5 and 4.8 stars
y['adj_pred']=1.5*(y.predi<1.5) + y.predi*(y.predi>=1.5)
y['adj_pred']= 4.8*(y.adj_pred>=4.8) + y.adj_pred*(y.adj_pred<4.8)

In [85]:
#Unadjusted RMSE
(sum((y['stars_revs']- y['predi'])**2)/len(y))**.5

1.1417513563620678

In [86]:
#Adjusted RMSE
(sum((y['stars_revs']- y['adj_pred'])**2)/len(y))**.5

1.141638832654151

In [0]:
y['star_est']=X['star_est']

In [93]:
#RMSE of my star estimate, not too bad
(sum((y['stars_revs']- y['star_est'])**2)/len(y))**.5

1.155854337524322

In [89]:
X.columns

Index(['elem', 'biz_num', 'latitude', 'longitude', 'review_count', 'stars_bus',
       'user_stars_mean', 'user_review_num', 'star_est'],
      dtype='object')

In [90]:
lr.coef_

array([[ 2.46083567e-06, -2.88954702e-05, -1.41767859e-02,
         2.37433911e-02, -3.12210925e-05,  2.32051226e-01,
         1.87303968e-01, -6.46832019e-05,  9.96828463e-01]])

In [91]:
lr.intercept_

array([0.9848939])

In [92]:
y.describe()

,stars_revs,predi,adj_pred,star_est
count,305173.000000,305173.000000,305173.000000,305173.000000
mean,3.649632,3.649632,3.649088,3.661754
std,1.263013,0.540000,0.537301,0.381767
min,1.000000,0.582799,1.500000,1.607964
25%,3.000000,3.313625,3.313625,3.420300
50%,4.000000,3.684526,3.684526,3.683335
75%,5.000000,4.028714,4.028714,3.933276
max,5.000000,5.320560,4.800000,4.807410
